# Tarea 2 PDM - Vicente Espinoza y Marcelo Vargas - Pregunta 2

In [1]:
!pip install pyspark

You should consider upgrading via the 'C:\Users\Vicente\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [6]:
from pyspark import SparkContext

# Crear un SparkContext
sc = SparkContext("local", "PageRank")

In [7]:
# Función para encontrar el costo del camino mas corto a cada nodo desde un nodo inicial
def single_shortest_path(nodo_inicial, nodes, edges):
  # Creamos una lista con tuplas (nodo, costo) con los nodos a los que les corresponde enviar un mensaje en la iteración actual
  nodos_actuales = [(nodo_inicial, 0)]

  # Creamos un RDD para guardar los costos acumulados para cada nodo
  acumulated_cost = nodes.map(lambda x: (x, 0) if x == nodo_inicial else (x, float('inf')))

  # Ejecutamos el algoritmo hasta que encuentre por primera los costos mas cortos
  while True:
    lista_auxiliar = []

    # Iteramos sobre los nodos a los que les corresponde enviar un mensaje
    for nodo, costo in nodos_actuales:
      # Encontramos los hijos del nodo actual
      hijos = edges.filter(lambda x: x[0] == nodo).map(lambda x: (x[1], x[2] + costo))
      # Actualizamos los costos acumulados
      new_acumulated_cost = acumulated_cost.leftOuterJoin(hijos).map(lambda x: check_min(x))
      # Verificamos que el nuevo RDD de costos acumulados es diferente al de la iteración anterior
      common_cost = acumulated_cost.intersection(new_acumulated_cost)
      #  Se termina el algoritmo si no hay cambios
      if common_cost.count() == acumulated_cost.count():
        return acumulated_cost

      print(acumulated_cost.collect())

      # Si hay cambios actualizamos la variable original de costos acumulados
      acumulated_cost = new_acumulated_cost

      lista_auxiliar += hijos.collect()
    nodos_actuales = lista_auxiliar

# Función que actualiza el costo de un nodo si es que es menor que el costo antiguo
def check_min(x):
  old_cost, new_cost = x[1][0], x[1][1]
  if new_cost is not None:
    return (x[0], min(old_cost, new_cost))
  else:
    return (x[0], old_cost)

In [8]:
# Creamos un RDD con los nodos y otro con sus aristas
nodes = sc.parallelize([1, 2, 3, 4])
edges = sc.parallelize([(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)])

# Establecemos un nodo de partida
nodo_inicial = 1

# Llamamos a la función
single_shortest_path(nodo_inicial, nodes, edges).collect()

[(1, 0), (2, inf), (3, inf), (4, inf)]
[(2, 10), (4, inf), (1, 0), (3, inf)]


[(3, 13), (4, 34), (1, 0), (2, 10)]